In [1]:
import pandas as pd

# Load your dataset (replace with your file path if not already loaded)
df = pd.read_csv("CEAS_08.csv")

# Display the first few rows of the 'sender' column
print(df['sender'].head())


0                     Young Esposito <Young@iworld.de>
1                         Mok <ipline's1983@icable.ph>
2    Daily Top 10 <Karmandeep-opengevl@universalnet...
3                   Michael Parker <ivqrnai@pobox.com>
4    Gretchen Suggs <externalsep1@loanofficertool.com>
Name: sender, dtype: object


In [3]:
df['sender_email'] = df['sender'].str.extract(r'<(.*?)>')


In [5]:
df['sender_domain'] = df['sender_email'].str.split('@').str[1]


In [9]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier

# Step 1: Extract sender email and domain
df['sender_email'] = df['sender'].str.extract(r'<(.*?)>')
df['sender_domain'] = df['sender_email'].str.split('@').str[1]
df = df.dropna(subset=['sender_domain'])

# Step 2: Vectorize the sender domains
vectorizer = TfidfVectorizer(max_features=1000)
X = vectorizer.fit_transform(df['sender_domain'])
y = df['label']

# Step 3: Train a simple model
model = RandomForestClassifier(random_state=42)
model.fit(X, y)

# Step 4: Explanation function
def explain_domain(domain):
    domain_vec = vectorizer.transform([domain])
    prediction = model.predict(domain_vec)[0]
    proba = model.predict_proba(domain_vec)[0][prediction]

    feature_names = vectorizer.get_feature_names_out()
    importance_scores = model.feature_importances_

    domain_parts = domain.split('.')
    keywords = []
    for part in domain_parts:
        if part in feature_names:
            idx = list(feature_names).index(part)
            score = importance_scores[idx]
            keywords.append((part, round(score, 4)))

    keywords = sorted(keywords, key=lambda x: -x[1])[:5]

    print("\n📨 Domain:", domain)
    print("🔎 Prediction:", "Phishing 🚨" if prediction == 1 else "Legitimate ✅")
    print("📊 Confidence Score:", round(proba, 4))
    print("📌 Influential keywords:", ", ".join([f"{w[0]} (score={w[1]})" for w in keywords]) if keywords else "No clear indicators")

# 🔍 Test on a few domains from your df
for domain in df['sender_domain'].sample(3, random_state=1):
    explain_domain(domain)



📨 Domain: caswellmassey.com
🔎 Prediction: Phishing 🚨
📊 Confidence Score: 0.9191
📌 Influential keywords: com (score=0.2414)

📨 Domain: dsi.unimi.it
🔎 Prediction: Legitimate ✅
📊 Confidence Score: 1.0
📌 Influential keywords: it (score=0.0042), dsi (score=0.0004), unimi (score=0.0003)

📨 Domain: factset.com
🔎 Prediction: Phishing 🚨
📊 Confidence Score: 0.9191
📌 Influential keywords: com (score=0.2414)


In [11]:
explain_domain("dsi.unimi.it")



📨 Domain: dsi.unimi.it
🔎 Prediction: Legitimate ✅
📊 Confidence Score: 1.0
📌 Influential keywords: it (score=0.0042), dsi (score=0.0004), unimi (score=0.0003)
